In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [7]:
csv_file = "netflix_titles.csv"
titles_df = pd.read_csv(csv_file)
titles_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [8]:
titles_df["date_added"] = pd.to_datetime(titles_df['date_added'])
titles_df['year_added'] = titles_df['date_added'].dt.year
titles_df['month_added'] = titles_df['date_added'].dt.month
## dropping 'duration' column as its not necessary for this analysis

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0


In [18]:
titles_df.drop(['director'], axis = 1, inplace = True)
titles_df.drop(['cast'], axis = 1, inplace = True)

In [21]:
titles_df.head()

,show_id,type,title,country,date_added,release_year,rating,listed_in,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,United States,2021-09-25,2020,PG-13,Documentaries,2021.0,9.0
1,s2,TV Show,Blood & Water,South Africa,2021-09-24,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2021.0,9.0
2,s3,TV Show,Ganglands,NaN,2021-09-24,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,NaN,2021-09-24,2021,TV-MA,"Docuseries, Reality TV",2021.0,9.0
4,s5,TV Show,Kota Factory,India,2021-09-24,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2021.0,9.0


In [20]:
titles_df.drop(['description'], axis = 1, inplace = True)

In [27]:
print(titles_df['country'].value_counts())

United States                             2818
India                                      972
United Kingdom                             419
Japan                                      245
South Korea                                199
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: country, Length: 748, dtype: int64


In [28]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/JohnSnowLabs/country-and-continent-codes-list/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)

    Continent_Name Continent_Code  \
0             Asia             AS   
1           Europe             EU   
2       Antarctica             AN   
3           Africa             AF   
4          Oceania             OC   
..             ...            ...   
257         Africa             AF   
258        Oceania             OC   
259           Asia             AS   
260           Asia             AS   
261           Asia             AS   

                                     Country_Name Two_Letter_Country_Code  \
0                Afghanistan, Islamic Republic of                      AF   
1                            Albania, Republic of                      AL   
2    Antarctica (the territory South of 60 deg S)                      AQ   
3        Algeria, People's Democratic Republic of                      DZ   
4                                  American Samoa                      AS   
..                                            ...                     ...   
257             

In [32]:
countries_expanded = titles_df['country'].str.split(',', expand=True)
countries_expanded.columns = ['Country'+str(i) for i in countries_expanded.columns]

countries_expanded_concat = pd.concat([titles_df,countries_expanded], axis=1)

year_country_produced_df = pd.melt(countries_expanded_concat, id_vars=['show_id','release_year'], value_vars=countries_expanded.columns, var_name='Country Number', value_name='Country Produced').dropna()

In [33]:
year_country_produced_df.head()

,show_id,release_year,Country Number,Country Produced
0,s1,2020,Country0,United States
1,s2,2021,Country0,South Africa
4,s5,2021,Country0,India
7,s8,1993,Country0,United States
8,s9,2021,Country0,United Kingdom


In [36]:
print(year_country_produced_df['Country Produced'].value_counts())

United States     3211
India             1008
United Kingdom     628
 United States     479
Canada             271
                  ... 
 Sri Lanka           1
Mozambique           1
 Palestine           1
 Cuba                1
 Mongolia            1
Name: Country Produced, Length: 197, dtype: int64


In [37]:
year_country_produced_df

,show_id,release_year,Country Number,Country Produced
0,s1,2020,Country0,United States
1,s2,2021,Country0,South Africa
4,s5,2021,Country0,India
7,s8,1993,Country0,United States
8,s9,2021,Country0,United Kingdom
...,...,...,...,...
78859,s8404,2014,Country8,Germany
85496,s6234,2017,Country9,Sweden
87666,s8404,2014,Country9,Netherlands
94303,s6234,2017,Country10,United States
